In [1]:
from embiggen.graph import GraphFactory

factory = GraphFactory(verbose=False)

In [2]:
path = "tests/data/rand_100nodes_5000edges.graph"

In [3]:
graph = factory.read_csv(
    path,
    edge_has_header=False,
    start_nodes_column=0,
    end_nodes_column=1,
    weights_column=2,
    return_weight=10,
    explore_weight=10
)
graph.random_walk(100, 100)

array([[ 0, 92, 54, ..., 89, 94, 87],
       [ 1, 15, 51, ..., 81, 57, 26],
       [ 2, 58, 19, ..., 10,  4, 95],
       ...,
       [96, 28, 64, ..., 80, 26, 29],
       [97, 51, 29, ...,  0, 30, 93],
       [98, 16, 38, ..., 47, 27, 38]])

In [4]:
%%timeit
graph = factory.read_csv(
    path,
    edge_has_header=False,
    start_nodes_column=0,
    end_nodes_column=1,
    weights_column=2,
    return_weight=10,
    explore_weight=10
)
graph.random_walk(100, 100)

504 ms ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
graph.random_walk(100, 100)

207 ms ± 4.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
type(results)

numpy.ndarray

In [10]:
import numpy as np
from numba import njit

@njit
def alias_draw(j: np.ndarray, q: np.ndarray) -> int:
    """Draw sample from a non-uniform discrete distribution using alias sampling.

    Parameters
    ----------
    j:np.ndarray,
        The mapping to the less probable binary outcome,
    q: np.ndarray
        Uniform distribution over binary outcomes

    Returns:
        index:int 
            index of random sample from non-uniform discrete distribution
    """
    # extract a random index for the mixture
    index = np.random.randint(0, len(q))
    # do the Bernulli trial
    if np.random.rand() < q[index]:
        # most probable case and fastest
        return index
    return j[index]

def no_njit_alias_draw(j: np.ndarray, q: np.ndarray) -> int:
    """Draw sample from a non-uniform discrete distribution using alias sampling.

    Parameters
    ----------
    j:np.ndarray,
        The mapping to the less probable binary outcome,
    q: np.ndarray
        Uniform distribution over binary outcomes

    Returns:
        index:int 
            index of random sample from non-uniform discrete distribution
    """
    # extract a random index for the mixture
    index = np.random.randint(0, len(q))
    # do the Bernulli trial
    if np.random.rand() < q[index]:
        # most probable case and fastest
        return index
    return j[index]


def new_alias_draw(samples_number:int=10000):
    samples = np.random.uniform(size=samples_number)
    last = samples[0]
    i = 1
    
    @njit
    def wrapped(j: np.ndarray, q: np.ndarray) -> int:
        """Draw sample from a non-uniform discrete distribution using alias sampling.

        Parameters
        ----------
        j:np.ndarray,
            The mapping to the less probable binary outcome,
        q: np.ndarray
            Uniform distribution over binary outcomes

        Returns:
            index:int 
                index of random sample from non-uniform discrete distribution
        """
        nonlocal i, last
        # extract a random index for the mixture
        index = int(samples[i-1] * len(q))
        new = samples[i]
        i = (i+1)%samples_number
        last = new
        # do the Bernulli trial
        if new < q[index]:
            # most probable case and fastest
            return index
        return j[index]
    
    return wrapped
    
wrapped_alias = new_alias_draw()

In [11]:
j = np.array([ 1,  0,  1,  2,  2,  2,  3,  6,  9,  6,  9, 10,  9, 10, 11, 11, 14,
        18, 16, 28, 30, 30, 30, 31, 34, 18, 25, 34, 26, 37, 28, 30, 31, 37,
        32, 37, 43, 34, 37, 44, 46, 46, 38, 42, 43, 50, 44, 50, 46, 50, 48,
        53, 53, 50, 53, 54, 55, 54, 56])
q = np.array([0.94942529, 1.        , 0.2679803 , 0.6909688 , 0.09688013,
        0.67816092, 0.37339901, 0.13563218, 0.65878489, 0.97832512,
        0.97504105, 0.80492611, 0.67816092, 0.27126437, 0.65418719,
        0.56190476, 0.33661741, 0.        , 0.17405583, 0.01937603,
        0.46502463, 0.93004926, 0.93004926, 0.32939245, 0.11625616,
        0.8952381 , 0.77142857, 0.91067323, 0.37635468, 0.03875205,
        0.67126437, 0.46666667, 0.54844007, 0.94942529, 0.32775041,
        0.94942529, 0.13563218, 0.42134647, 0.99178982, 0.67816092,
        0.89129721, 0.67816092, 0.73234811, 0.55041051, 0.90344828,
        0.        , 0.96584565, 0.93004926, 0.82692939, 0.65878489,
        0.25747126, 0.4456486 , 0.69753695, 0.78916256, 0.90213465,
        0.77077176, 0.53070608, 0.71691297, 0.17438424])

In [12]:
%%timeit
alias_draw(j, q)

437 ns ± 5.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%%timeit
wrapped_alias(j, q)

406 ns ± 5.44 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
%%timeit
no_njit_alias_draw(j, q)

7.06 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
